## 방법 1. TF-IDF + TruncatedSVD(차원 축소) (추천)
- 질문 컬럼을 벡터화 하고 차원축소

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBRanker
from sklearn.metrics import ndcg_score

In [2]:
df = pd.read_csv(".\\csv_파일\\3000개_질문.csv")

In [3]:
# llm_rank를 관련성 점수로 변환
# 1위 -> 5점, 2위 -> 3점, 3위 -> 2점, 나머지는 0점 (역순 변환)
def convert_rank_to_score(rank):
    if rank == 1: return 30
    elif rank == 2: return 20
    elif rank == 3: return 17
    elif rank == 4 : return 13
    elif rank == 5 : return 10
    elif rank == 6 : return 7
    elif rank == 7 : return 5
    else: return 0

# 정답(y)은 정수형으로 변환 (0~10 단계로 변환)
df['relevance_label'] = df['llm_rank'].apply(convert_rank_to_score)

---
### 데이터 분할

In [4]:
# 1. 사용자 유형별로 질문(Group) 리스트 추출
queries_0 = df[df['사용자 유형'] == 0]['질문'].unique() # 입문자 질문들
queries_1 = df[df['사용자 유형'] == 1]['질문'].unique() # 기존 팬 질문들

# 2. 각 유형 내에서 8:2로 학습/테스트 질문 분리
# (전체 데이터에서도 8:2 비율이 유지되면서 테스트셋 구성됨)
train_q0, test_q0 = train_test_split(queries_0, test_size=0.2, random_state=42)
train_q1, test_q1 = train_test_split(queries_1, test_size=0.2, random_state=42)

# 3. 최종 학습 및 테스트 데이터 생성
train_df = df[df['질문'].isin(list(train_q0) + list(train_q1))].sort_values('질문')
test_df  = df[df['질문'].isin(list(test_q0) + list(test_q1))].sort_values('질문')

# 4) ✅ 인코더는 train에만 fit 하고, test는 transform만!
#    + test에 새로운 팀이 나와도 깨지지 않게 unknown_value=-1 처리
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

train_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.fit_transform(
    train_df[['기존 응원 팀', '매칭팀']]
)

test_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.transform(
    test_df[['기존 응원 팀', '매칭팀']]
)

# ----------------
# 5) 질문 벡터화(TF-IDF) (train에만 fit)
# ----------------
tfidf = TfidfVectorizer(
    ngram_range = (1, 2),  # 1-gram(단어) + 2-gram(단어쌍)까지 반영
    min_df = 2,            # 2개 미만 문서에만 등장하는 희귀 토큰 제거
    max_df = 0.9           # 너무 자주 등장하는 토큰 제거(잡음 감소)
)

Xq_train_tfidf = tfidf.fit_transform(train_df['질문'])  # train 질문으로만 단어사전/가중치 학습 + 변환
Xq_test_tfidf  = tfidf.transform(test_df['질문'])       # test 질문은 학습 없이 변환만

# ----------------
# 6) 차원 축소(SVD) (train에만 fit)
# ----------------
svd = TruncatedSVD(n_components=50, random_state=42)     # TF-IDF 고차원 → 50차원으로 압축

Xq_train_svd = svd.fit_transform(Xq_train_tfidf)         # train에서만 축소기 학습 + 변환
Xq_test_svd  = svd.transform(Xq_test_tfidf)              # test는 변환만

# ----------------
# 7) 축소된 질문 벡터를 DataFrame으로 만들어서 붙일 준비
# ----------------
q_cols = [f"q_svd_{i}" for i in range(Xq_train_svd.shape[1])]  # q_svd_0 ~ q_svd_49 컬럼명 생성

q_train_df = pd.DataFrame(Xq_train_svd, columns=q_cols, index=train_df.index)  # train 질문 벡터 DF
q_test_df  = pd.DataFrame(Xq_test_svd,  columns=q_cols, index=test_df.index)   # test 질문 벡터 DF

# ----------------
# 8) 최종 features 구성 (질문 원문은 제외, q_svd_*만 포함)
# ----------------
features = [
    '사용자 유형',
    'sbert_score',
    'n2v_score',
    'vector_score' ,
    '기존_응원팀_encoded' ,
    '매칭팀_encoded'
]  # 기존에 쓰던 수치형 피처들

X_train = train_df[features].copy()  # train에서 기본 피처만 뽑음
X_test  = test_df[features].copy()   # test에서 기본 피처만 뽑음

X_train = pd.concat([X_train, q_train_df], axis=1)  # train에 질문 임베딩(q_svd_*) 붙이기
X_test  = pd.concat([X_test,  q_test_df],  axis=1)  # test에 질문 임베딩(q_svd_*) 붙이기

# ----------------
# 9) 라벨 생성 (너가 쓰던 방식 유지)
# ----------------
y_train = train_df['llm_rank'].apply(lambda x: 12 - x if x <= 11 else 0)  # rank 1→3, 2→2, 3→1, 그 외→0
y_test  = test_df['llm_rank'].apply(lambda x: 12 - x if x <= 11 else 0)   # test도 동일 변환

# ----------------
# 10) 그룹(쿼리별 문서 수) 계산 (질문 기준)
# ----------------
group_train = train_df.groupby('질문').size().to_list()  # train에서 질문별 후보 개수 리스트
group_test  = test_df.groupby('질문').size().to_list()   # test에서 질문별 후보 개수 리스트

# ----------------
# 11) (선택) sanity check 출력
# ----------------
print("X_train shape:", X_train.shape)  # train 피처 차원 확인
print("X_test  shape:", X_test.shape)   # test 피처 차원 확인
print("num train groups:", len(group_train), " / sum:", sum(group_train))  # 그룹 수, 총 행 수 확인
print("num test groups :", len(group_test),  " / sum:", sum(group_test))   # 그룹 수, 총 행 수 확인

X_train shape: (2903, 56)
X_test  shape: (917, 56)
num train groups: 72  / sum: 2903
num test groups : 19  / sum: 917


---
### 모델 학습

In [5]:
model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg@3",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [6]:
model.fit(X_train, y_train, group = group_train)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg@3'


🔍 LightGBM 로그 분석 결과
1. `No further splits with positive gain` (가장 많이 뜬 경고)
    - 의미: 나무를 더 잘게 쪼개서 학습하려고 했는데, 더 이상 쪼개봤자 성능 향상에 도움이 안 된다고 판단해서 멈춘 거야.

    - 원인: 우리 데이터(약 1,100행)가 머신러닝 기준으로는 아주 큰 편은 아니고, 피처(4개)가 명확해서 모델이 금방 정답 패턴을 찾았기 때문이야.

    - 결과: 이건 에러가 아니라 모델이 과적합(Overfitting)되지 않게 스스로 공부를 적당히 멈춘 똑똑한 상태라고 보면 돼!

2. `Found whitespace in feature_names`
    - 의미: 피처 이름 중에 '사용자 유형'처럼 공백(띄어쓰기)이 있는 게 있다는 뜻이야.

    - 결과: LightGBM이 알아서 사용자_유형처럼 언더바(_)로 바꿔서 처리했으니 아무 문제 없어.

3. `Found 'eval_at' in params`
    - 의미: eval_at 파라미터가 중복으로 설정됐다는 경고야.

    - 결과: 모델 생성할 때 넣은 설정값이 우선 적용됐다는 뜻이니 이것도 안심해도 돼.

In [7]:
scores = model.predict(X_test)

---
### 모델 검증 $\rightarrow$ 실제 추천 리스트로 변환

In [8]:
# 모델이 뱉은 점수를 test_df에 다시 붙여서, 질문별로 어떤 팀이 1등인지 눈으로 확인

# 1. 테스트 데이터 복사본에 예측 점수 추가
test_results = test_df.copy()
test_results['predicted_score'] = scores

# 2. 질문별로 점수가 높은 순(내림차순)으로 정렬
# '질문'과 'predicted_score' 순으로 정렬
test_results = test_results.sort_values(by=['질문', 'predicted_score'], ascending=[True, False])

# 3. 각 질문에서 모델이 뽑은 Top 3 팀 확인
final_top3 = test_results.groupby('질문').head(3)
print(final_top3[['질문', '매칭팀', 'predicted_score', 'llm_rank']])

# '사용자 유형'별로 필터링해서 저장해두면 나중에 피그마 UI에 넣기 좋아!
# final_top3.to_csv("final_recommendation_results.csv", index=False, encoding="utf-8-sig")

                                                     질문         매칭팀  \
2922          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.         페라리   
3729          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.         페라리   
2949          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.     삼성 라이온즈   
591                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.         페라리   
597                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.   전북 현대 모터스   
594                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.  수원 삼성 블루윙즈   
2792           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.         페라리   
3600           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.         페라리   
3588           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.         리버풀   
3341                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.         페라리   
2527                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.         페라리   
2537                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.   전북 현대 모터스   
1789                나는 F1는 처음인데, 우승권 유지 성향이 강한 팀을 추천해줘.         페라리   
1800  

### NDCG
: 추천된 팀들의 **순서(Ranking)**가 얼마나 정확한지 측정하는 지표.
</br> 1등 팀을 1등으로 맞추는 게 2등으로 맞추는 것보다 더 높은 점수를 받음.

- 0.7 이상: 모델이 꽤 영리하게 추천을 잘해줌

- 0.9 이상: LLM이 매긴 순위랑 거의 똑같이 맞추고 있음

- 만약 점수가 너무 낮게(0.5 이하) 나온다면?
</br>: 모델이 아직 '사용자 유형'이나 '벡터 스코어'의 특징을 제대로 못 잡았다는 신호
</br>$\Rightarrow$ learning_rate를 조절하거나 피처를 더 추가하는 파라미터 튜닝 단계 진행

In [9]:
# 모델 성능 평가하기 (NDCG 지표 확인)

from sklearn.metrics import ndcg_score
import numpy as np

# 1. 질문별 NDCG 점수를 저장할 리스트
ndcg_list = []

# 2. 질문(그룹)별로 루프를 돌며 계산
for qid in test_results['질문'].unique():
    # 해당 질문에 속한 데이터만 추출
    group_data = test_results[test_results['질문'] == qid]
    
    # 실제 정답(llm_rank 기반 점수)과 모델의 예측 점수 추출
    # ndcg_score는 2차원 배열 형식을 원하므로 reshape(1, -1)을 해줘.
    y_true = np.array([group_data['relevance_label']]).reshape(1, -1)
    y_score = np.array([group_data['predicted_score']]).reshape(1, -1)
    
    # NDCG@3 계산 (상위 3개 팀의 순위가 얼마나 정확한가?)
    # 데이터가 3개보다 적을 경우를 대비해 k값을 조절해.
    k = min(len(group_data), 3)
    score = ndcg_score(y_true, y_score, k=k)
    ndcg_list.append(score)

# 3. 최종 평균 NDCG 출력
avg_ndcg = np.mean(ndcg_list)
print(f" 최종 모델 성능 (Average NDCG@3): {avg_ndcg:.4f}")


 최종 모델 성능 (Average NDCG@3): 0.7103


In [10]:
from itertools import product
from xgboost import XGBRanker
from sklearn.metrics import ndcg_score
import numpy as np

param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [4, 6],
    "n_estimators": [200, 400],
    "subsample": [0.8, 1.0]
}

best_score = -1
best_params = None

for lr, md, ne, ss in product(
    param_grid["learning_rate"],
    param_grid["max_depth"],
    param_grid["n_estimators"],
    param_grid["subsample"]
):
    model = XGBRanker(
        objective="rank:ndcg",
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=ss,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        group = group_train
    )

    preds = model.predict(X_test)

    score = ndcg_score(
        [y_test],
        [preds]
    )

    if score > best_score:
        best_score = score
        best_params = {
            "learning_rate": lr,
            "max_depth": md,
            "n_estimators": ne,
            "subsample": ss
        }

print("Best NDCG:", best_score)
print("Best Params:", best_params)


Best NDCG: 0.9658484627773642
Best Params: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 400, 'subsample': 1.0}
